# Khám phá mối quan hệ trong dữ liệu

## Import các thư viện cần thiết

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from wordcloud import WordCloud
import plotly.express as px


pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)

## Đọc dữ liệu từ file csv vào dataframe

In [2]:
data = pd.read_csv("../datasets/data_preprocess.csv", index_col = 0)
data.head()

film_code  cinema_code  total_sales  tickets_sold  tickets_out  show_time  \
0       1492          304      3900000            26            0          4   
1       1492          352      3360000            42            0          5   
2       1492          489      2560000            32            0          4   
3       1492          429      1200000            12            0          1   
4       1492          524      1200000            15            0          3   

   occu_perc  ticket_price  ticket_use    capacity        date  month  \
0       4.26      150000.0          26  610.328638  2018-05-05      5   
1       8.08       80000.0          42  519.801980  2018-05-05      5   
2      20.00       80000.0          32  160.000000  2018-05-05      5   
3      11.01      100000.0          12  108.991826  2018-05-05      5   
4      16.67       80000.0          15   89.982004  2018-05-05      5   

   quarter  day  
0        2    5  
1        2    5  
2        2    5  
3        2    5  
4        2    5

In [3]:
data['date'] = pd.to_datetime(data['date'])
data[['film_code', 'cinema_code']] = data[['film_code', 'cinema_code']].astype(str)

In [4]:
data

film_code cinema_code  total_sales  tickets_sold  tickets_out  \
0           1492         304      3900000            26            0   
1           1492         352      3360000            42            0   
2           1492         489      2560000            32            0   
3           1492         429      1200000            12            0   
4           1492         524      1200000            15            0   
...          ...         ...          ...           ...          ...   
142519      1569         495      1320000            22            0   
142520      1569         474      1200000            15            0   
142521      1569         524      1060000             8            0   
142522      1569         529       600000             5            0   
142523      1569         486       250000             5            0   

        show_time  occu_perc  ticket_price  ticket_use    capacity       date  \
0               4       4.26      150000.0          26  610.328638 2018-05-05   
1               5       8.08       80000.0          42  519.801980 2018-05-05   
2               4      20.00       80000.0          32  160.000000 2018-05-05   
3               1      11.01      100000.0          12  108.991826 2018-05-05   
4               3      16.67       80000.0          15   89.982004 2018-05-05   
...           ...        ...           ...         ...         ...        ...   
142519          2       3.86       60000.0          22  569.948187 2018-11-04   
142520          1      65.22       80000.0          15   22.999080 2018-11-04   
142521          3       9.20      132500.0           8   86.956522 2018-11-04   
142522          2       5.00      120000.0           5  100.000000 2018-11-04   
142523          1       1.79       50000.0           5  279.329609 2018-11-04   

        month  quarter  day  
0           5        2    5  
1           5        2    5  
2           5        2    5  
3           5        2    5  
4           5        2    5  
...       ...      ...  ...  
142519     11        4    4  
142520     11        4    4  
142521     11        4    4  
142522     11        4    4  
142523     11        4    4  

[142418 rows x 14 columns]

### Số lượng các bộ phim được chiếu theo thời gian

**Trường dữ liệu**
- `date`, `film_code`

**Lý do chọn biểu đồ**
- Biểu đồ tốt nhất để trực quan dữ liệu time series là line chart. Tuy nhiên, do trong dataset của chúng ta có những ngày không có dữ liệu, thế nên không thể trực quan theo hằng ngày bằng line chart được, lúc này muốn thể hiện đầy đủ dữ liệu lên biểu đồ thì scatter là lựa chọn hợp lý nhất. 
- Vì dữ liệu trong ngày có thể thiếu, nhưng dữ liệu trong tháng thì luôn đầy đủ nên ở biểu đồ thứ 2 có thể sử dụng line chart. Hơn nữa, thời gian một bộ phim được chiếu đều rất lâu, có thể tính theo đơn vị tháng, thế nên việc nhóm dữ liệu lại theo tháng cũng là hợp lý. 

Đầu tiên ta vẽ scatter plot trực quan số các bộ phim khác nhau được chiếu trong ngày.

In [74]:
n_film_by_date = data.groupby('date')['film_code'].nunique().reset_index()

In [81]:
fig = px.scatter(n_film_by_date, x='date', y='film_code', 
              title='Số lượng các bộ phim được chiếu trong ngày theo thời gian')

fig.update_layout(xaxis_title='thời gian',
                  yaxis_title='Số lượng')

fig.show()

Tiếp theo chúng ta vẽ line chart để trực quan số phim được chiếu trong tháng theo thời gian, line chart giúp ta dễ dàng rút ra được sự biến đổi của dữ liệu theo thời gian.

In [77]:
n_film_by_month = data.groupby('month')['film_code'].nunique().reset_index()

In [83]:
fig = px.line(n_film_by_month, x='month', y='film_code', 
              title='Số lượng các bộ phim được chiếu trong tháng theo thời gian')

fig.update_layout(xaxis_title='tháng',
                  yaxis_title='Số lượng')

fig.show()

**Nhận xét**
- Việc sử dụng plotly để trực quan giúp ta có thể tương tác được với biểu đồ, chọn các mốc thời gian khác nhau, gúp quan sát chi tiết hơn.
- Số phim được chiếu trong tháng theo thời gian là không giống nhau.
- Ở đoạn khoảng tháng 2,3 số lượng ít là do dữ liệu của chúng ta không đầy đủ (Dữ liệu của chúng ta bắt đầu từ cuối tháng 2, và dữ liệu của tháng 3 cũng chỉ có nửa tháng sau) 
- Từ tháng 4 đến tháng 6 số lượng phim ít, nhưng tăng dần. Chứng tỏ rất có 1 phim rất hay được chiếu trong tháng 4, nó kéo dài đến tháng 6. Vì phim này có nhiều người đi xem nên các rạp tập trung mở nhiều suất chiếu cho phim này, các phim khác cũng chiếu ở thời gian khác nếu không muốn bị ít khán giả do họ đều đi xem phim hay kia rồi. Các tháng này không có nhiều phim để xem, đa phần đều đi xem phim là xem bộ phim kia.
- Từ tháng 7 trở đi, ở scatter plot chúng ta có thể thấy dữ liệu biến động khá nhiều (ở các tháng trước thì rất nhiều ngày liên tiếp nhau có cùng giá trị), chứng tỏ rất thường xuyên có việc phim cũ ngừng chiếu, phim mới bắt đầu chiếu. Các tháng này có rất nhiều phim để xem.

**Kiểm chứng phim nhiều lượt xem nhất có phải chiếu ở tháng 4 hay không**

In [107]:
# số lượng vé bán của mỗi phim
sales_data = data.groupby('film_code')['tickets_sold'].sum()

# film có số vé bán được nhiều nhất
film_max = sales_data[sales_data == max(sales_data)].index.values[0]

# ngày bắt đầu chiếu phim bán được chiếu vé nhất (được xem nhiều nhất)
data[data['film_code'] == film_max]['date'].min()

Timestamp('2018-07-04 00:00:00')

### Số suất chiếu phim theo thời gian của các rạp

**Trường dữ liệu**
- `date`, `show_time`

**Lý do chọn biểu đồ**
- Biểu đồ tốt nhất để trực quan dữ liệu time series là line chart. Tuy nhiên, do trong dataset của chúng ta có những ngày không có dữ liệu, thế nên không thể trực quan theo hằng ngày bằng line chart được, lúc này scatter là lựa chọn hợp lý nhất. Chúng ta cũng có thể xem số suất chiếu phim trong ngày theo thời gian qua scatter plot.
- Vì dữ liệu trong ngày có thể thiếu, nhưng dữ liệu trong tuần thì luôn đầy đủ, ở biểu đồ thứ 2 chúng ta có thể trực quan số suất chiếu trong tuần theo thời gian bằng line chart.

Đầu tiên chúng ta vẽ scatter plot để trực quan số suất chiếu phim trong ngày theo thời gian, scatter plot có thể thể hiện đầy đủ dữ liệu lên biểu đồ.

In [84]:
n_show_time_by_date = data.groupby('date')['show_time'].sum().reset_index()

In [97]:
fig = px.scatter(n_show_time_by_date, x='date', y='show_time', 
              title='Số lượng suất chiếu phim trong ngày theo thời gian')

fig.update_layout(xaxis_title='thời gian',
                  yaxis_title='Số lượng')

fig.show()

Tiếp theo chúng ta vẽ line chart để trực quan số suất chiếu phim trong tuần theo thời gian, line chart giúp ta dễ dàng rút ra được sự biến đổi của dữ liệu theo thời gian.

In [92]:
n_show_time_by_week = data.groupby('date')['show_time'].sum().resample('W').sum().reset_index()

In [96]:
fig = px.line(n_show_time_by_week, x='date', y='show_time', 
              title='Số lượng suất chiếu phim trong tuần theo thời gian')

fig.update_layout(xaxis_title='thời gian',
                  yaxis_title='Số lượng')

fig.show()

**Nhận xét**
- Kết hợp với biểu đồ của câu hỏi trước (số lượng các bộ phim chiếu trong ngày), ta có thể thấy mặc dù số lượng phim được chiếu trong 1 ngày là nhiều ít tùy lúc khác nhau, nhưng không vì số lượng phim mà nó ảnh hưởng đến số lượng suất chiếu phim của rạp. 
- Qua biểu đồ chúng ta có thể thấy rằng mặc kệ phim this phim that, phim nhiều hay ít thì tổng số suất chiếu trong 1 ngày của tất cả các rạp đều duy trì ở mức khoảng 2500 suất trở lên. Ngoại trừ các outlier có thể là do dữ liệu của chúng ta hôm đó cập nhật thiếu hay sự cố gì đó chẳng hạng.
- Dường như rạp phim luôn cố gắng hoạt động hết mức công suất, luôn duy trì số lượng suất chiếu phim trong ngày mặc dù chưa biết có nhiều người đi xem hay không. Họ không vì số lượng phim hay số lượng người đi xem phim mà tăng hay giảm số suất chiếu phim.